In [1]:
from __future__ import print_function

from imutils.object_detection import non_max_suppression
from imutils import paths

from FileSelector import *
from Recognizer import *

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
import matplotlib.animation as animation

from matplotlib.patches import Rectangle
from matplotlib.animation import FuncAnimation
from matplotlib import style

import imageio

from PIL import Image
import imutils
import cv2

import json
import os

from os import listdir
from os.path import isfile, join

from pathlib import Path
import numpy as np

In [2]:
def get_frames_at_rate(video, rate=10, max_num_frames=100):
    frames = []
    capture = cv2.VideoCapture(my_video)
    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    captured = 0
    
    while captured < max_num_frames and captured * rate < frame_count:
        capture.set(cv2.CAP_PROP_POS_FRAMES, rate * captured)
        _, frame = capture.read()
        frames.append(frame)
        
        captured += 1
    return frames

In [18]:
def detect_pedestrians(frames):
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    
    new_frames = []
    
    for frame in frames:
        (rects, weights) = hog.detectMultiScale(frame, winStride=(4, 4), padding=(8, 8), scale=1.05)
        for (x, y, w, h) in rects:
            new_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            #cv2.rectangle(new_frame, (x, y), (x + w, y + h), (0, 0, 255), 0)
            
        rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
        pick = non_max_suppression(rects, probs=None, overlapThresh=0.50)
        
        for (xA, yA, xB, yB) in pick:
            cv2.rectangle(new_frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
    
        new_frames.append(new_frame)
    
    return new_frames

### ! Unfortunately, it seems we need to save the frames as images for the plate recognizer to process them :/ !

In [19]:
def save_frames(frames, directory='..\\temp\\'):
    for i in range(len(frames)):
        print(directory + str(i).zfill(3) + '.jpg')
        cv2.imwrite(directory + str(i).zfill(3) + '.jpg', frames[i])

In [20]:
def get_boxes(node):
    boxes = []
    for result in node[0]['results']:
        box = result['box']
        pt_1, pt_2 = (box['xmin'], box['ymin']), (box['xmax'], box['ymax'])
        boxes.append((pt_1, pt_2))
        
    return boxes

In [21]:
def detect_plates(directory):
    files = [join(directory, f) for f in listdir(directory) if isfile(join(directory, f))]
    
    frames = []
    
    for file in files:
        #print(file)
        node = recognize_img(file)
        boxes = get_boxes(node)

        for box in boxes:
            frame = cv2.imread(file, 1)
            if len(box) !=0:
                new_frame = frame #cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                cv2.rectangle(new_frame, box[0], box[1], (255, 255, 0), 2)
            
            frames.append(new_frame)
            
        #print('A new frame processed')
    return frames

In [22]:
def display_gif(fn):
    from IPython import display
    return display.HTML('<img src="{}">'.format(fn))

In [23]:
selector = SelectFilesButton()

In [24]:
selector

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [25]:
my_video = selector.files[0]

In [26]:
my_frames = get_frames_at_rate(my_video, 10, 30)

In [27]:
# I have used this because I keep getting issues with relative paths, please change it!
temp = 'C:\\Users\\admin\\Documents\\plate_project\\temp\\'
save_frames(my_frames, temp)

C:\Users\admin\Documents\plate_project\temp\000.jpg
C:\Users\admin\Documents\plate_project\temp\001.jpg
C:\Users\admin\Documents\plate_project\temp\002.jpg
C:\Users\admin\Documents\plate_project\temp\003.jpg
C:\Users\admin\Documents\plate_project\temp\004.jpg
C:\Users\admin\Documents\plate_project\temp\005.jpg
C:\Users\admin\Documents\plate_project\temp\006.jpg
C:\Users\admin\Documents\plate_project\temp\007.jpg
C:\Users\admin\Documents\plate_project\temp\008.jpg
C:\Users\admin\Documents\plate_project\temp\009.jpg
C:\Users\admin\Documents\plate_project\temp\010.jpg
C:\Users\admin\Documents\plate_project\temp\011.jpg
C:\Users\admin\Documents\plate_project\temp\012.jpg
C:\Users\admin\Documents\plate_project\temp\013.jpg
C:\Users\admin\Documents\plate_project\temp\014.jpg
C:\Users\admin\Documents\plate_project\temp\015.jpg
C:\Users\admin\Documents\plate_project\temp\016.jpg
C:\Users\admin\Documents\plate_project\temp\017.jpg
C:\Users\admin\Documents\plate_project\temp\018.jpg
C:\Users\adm

In [28]:
my_new_frames = detect_plates(temp)

In [29]:
my_new_frames = detect_pedestrians(my_new_frames)

In [30]:
# save the gif file
temp_file = 'assets\\my_car.gif'
imageio.mimsave(temp_file, my_new_frames, duration=0.2)

In [31]:
display_gif(temp_file)

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(cv2.cvtColor(my_new_frames[0], cv2.COLOR_BGR2RGB))